In [8]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.constants import year

In [9]:
class EyeDetector:
    def __init__(self, ear_thresh=0.24, max_count=30):
        self.index_left_eye = [33, 160, 158, 133, 153, 144]
        self.index_right_eye = [362, 385, 387, 263, 373, 380]
        self.ear_thresh = ear_thresh
        self.count = 0
        self.max_count = max_count
        self.mp_face_mesh = mp.solutions.face_mesh
        self.openEye=None

    def __eye_aspect_ratio(self, coordinates):
        d_A = np.linalg.norm(np.array(coordinates[1]) - np.array(coordinates[5]))
        d_B = np.linalg.norm(np.array(coordinates[2]) - np.array(coordinates[4]))
        d_C = np.linalg.norm(np.array(coordinates[0]) - np.array(coordinates[3]))
        return (d_A + d_B) / (2 * d_C)
    
    # this method loop
    def detect(self, frame, face_mesh):
        height, width, _ = frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)

        coordinates_left_eye = []
        coordinates_right_eye = []
        
        if results.multi_face_landmarks is not None:
            for face_landmarks in results.multi_face_landmarks:
                for index in self.index_left_eye:
                    x = int(face_landmarks.landmark[index].x * width)
                    y = int(face_landmarks.landmark[index].y * height)
                    coordinates_left_eye.append([x, y])
                    cv2.circle(frame, (x, y), 2, (0, 255, 255), 1)
                    cv2.circle(frame, (x, y), 1, (128, 0, 250), 1)
                for index in self.index_right_eye:
                    x = int(face_landmarks.landmark[index].x * width)
                    y = int(face_landmarks.landmark[index].y * height)
                    coordinates_right_eye.append([x, y])
                    cv2.circle(frame, (x, y), 2, (128, 0, 250), 1)
                    cv2.circle(frame, (x, y), 1, (0, 255, 255), 1)

            ear_left_eye = self.__eye_aspect_ratio(coordinates_left_eye)
            ear_right_eye = self.__eye_aspect_ratio(coordinates_right_eye)
            ear = (ear_left_eye + ear_right_eye) / 2

            if ear > self.ear_thresh and self.count <= self.max_count:
                self.count += 1
                self.openEye = True
            elif ear < self.ear_thresh and self.count >= -self.max_count:
                self.count -= 1
                self.openEye = False
        return frame
    
    # true if 'despierto' else false
    def isAwake(self):
        return self.count>0
    
    # true if open eye else false
    def isOpenEye(self):
        return self.openEye


    def get_info(self):
        status = "Despierto" if self.isAwake() else "Dormido"
        open = "Sí" if self.isOpenEye() else "No"
        return f"EYE=>Estado: {status}, Ojos Abiertos: {open}, Contador: {self.count}"

class MotionDetector:
    def __init__(self, area_x=100, area_y=100, area_w=200, area_h=200, min_contour_area=5000, max_count=30):
        self.area_x = area_x
        self.area_y = area_y
        self.area_w = area_w
        self.area_h = area_h
        self.min_contour_area = min_contour_area
        self.count = 0
        self.max_count = max_count
        self.is_move = False

    def __intersect_rect(self, x1, y1, w1, h1, x2, y2, w2, h2):
        return not (x1 + w1 < x2 or x2 + w2 < x1 or y1 + h1 < y2 or y2 + h2 < y1)

    def __is_contained(self, x, y, w, h):
        return (x >= self.area_x and y >= self.area_y and x + w <= 
                self.area_x + self.area_w and y + h <= self.area_y + self.area_h)
    
    def updatePosition(self, pos_x, pos_y, area_w, area_h):
        self.area_x = pos_x
        self.area_y = pos_y
        self.area_w = area_w
        self.area_h = area_h

    def detect(self, frame1, frame2):
        diff = cv2.absdiff(frame1, frame2)
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
        dilated = cv2.dilate(thresh, None, iterations=3)
        contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        cv2.rectangle(frame1, (self.area_x, self.area_y),
                      (self.area_x + self.area_w, self.area_y + self.area_h),
                      (0, 0, 255), 2)

        for contour in contours:
            if cv2.contourArea(contour) < self.min_contour_area:
                continue

            x, y, w, h = cv2.boundingRect(contour)

            if self.__intersect_rect(x, y, w, h, self.area_x, self.area_y, self.area_w, self.area_h):
                cv2.rectangle(frame1, (x, y), (x + w, y + h), (0, 255, 0), 2)
                self.is_move = True
                if self.count <= self.max_count:
                    self.count += 1.5

        if not self.is_move and self.count >= -self.max_count:
            self.count -= 1
        if self.is_move:
            self.is_move = False
        return frame1
    
    def isAwake(self):
        return self.count>0
    
    #DEPRECATED NOT VALID
    def isMove(self):
        return self.is_move

    def get_info(self):
        status = "Despierto" if self.isAwake() else "Dormido"
        move = "Sí" if self.isMove() else "No"
        return f"MOTION=>Estado: {status}, Movimiento: {move}, Contador: {self.count}"


In [10]:
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier('./models/haarcascade_frontalface_default.xml')
eye_detector = EyeDetector(ear_thresh=0.24, max_count=30)
motion_detector = MotionDetector()

with eye_detector.mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1) as face_mesh:
    while True:
        ret, frame1 = cap.read()
        ret, frame2 = cap.read()
        if not ret:
            break

        #frame1 = cv2.flip(frame1, 1)
        face_rects = face_cascade.detectMultiScale(frame1, scaleFactor=1.3,minNeighbors=3)
        if len(face_rects) > 0:
            x, y, w, h = face_rects[0]
            motion_detector.updatePosition(x, y, w, h)
        else:
            x, y, w, h = 0,0,0,0
            motion_detector.updatePosition(x, y, w, h)
            
        frame1 = eye_detector.detect(frame1, face_mesh)
        frame1 = motion_detector.detect(frame1, frame2)
        #print(eye_detector.get_info()) #This method detect if sleep
        #print(motion_detector.get_info())
        

        cv2.imshow('frame', frame1)
        key = cv2.waitKey(20) & 0xFF
        if key == 27:  # Tecla ESC para salir
            break

cap.release()
cv2.destroyAllWindows()


libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 